<a href="https://colab.research.google.com/github/Saif-Al-Zarrar/FamilyMemberClassifer/blob/master/VGG-Incorrrect(BinaryCE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import math

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
IMAGE_SIZE = [224, 224]

#Give dataset path
train_path = '/content/gdrive/My Drive/ExtendedFamilyClassifier/Dataset/Train'
validation_path = '/content/gdrive/My Drive/ExtendedFamilyClassifier/Dataset/Validation'


vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)



# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False



# useful for getting number of classes
folders = glob('/content/gdrive/My Drive/ExtendedFamilyClassifier/Dataset/Train/*')
print(len(folders))


x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()



from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])




# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


# Data Augmentation
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')



train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

validation_set = validation_datagen.flow_from_directory(validation_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')



from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# learning rate schedule
def step_decay(epoch):
    initial_lrate = .5
    drop = 0.25
    epochs_drop = 4.0
    schedule = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return schedule
    
lr_scheduler = LearningRateScheduler(step_decay, verbose=0)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               monitor='val_loss',
                               cooldown=1,
                               patience=5,
                               min_lr=0.5e-6,
                               min_delta=0.5)

#num_epochs = 1000
#num_batch_size = 32

ES = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=18, baseline=0.95, min_delta=.5,)

checkpoint = ModelCheckpoint(filepath='/content/gdrive/My Drive/ExtendedFamilyClassifier/Models/VGG/VGG_Model1-{epoch:02d}-{val_accuracy:.2f}.hdf5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint, lr_reducer, lr_scheduler, ES]

start = datetime.now()



model.fit_generator(
  train_set,
  validation_data=validation_set,
  epochs=25,
  steps_per_epoch=5,
  validation_steps=32,
    callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)


6
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0 

In [ ]:
import os

# Directory Path-Test
Dir="/content/gdrive/My Drive/ExtendedFamilyClassifier/Dataset/Test"

# Directory Path-Train
# Dir="/content/gdrive/My Drive/ExtendedFamilyClassifier/Dataset/Train&Validation"

# Code to get list of sub directories in the parent directory
sub_dir_name_list = os.listdir(Dir)
#sub_dir_name_list
 
# Create paths to subdirectories
sub_dir_path_list = [os.path.join(Dir, sub_dir_name) for sub_dir_name in sub_dir_name_list]
#sub_dir_path_list

# Code to alpbetical create prefix for file names for files in sub folders
count=1
while True:
    file_name_prefix_list = [sub_dir_name[0:count] for sub_dir_name in sub_dir_name_list]
    if len(file_name_prefix_list) == len(set(file_name_prefix_list)):
        break
    else:
        count=count + 1
sub_dir_name_list.sort()
sub_dir_path_list.sort()
file_name_prefix_list.sort()

In [ ]:
import os
from keras.models import load_model
import numpy as np
from keras.preprocessing import image


Accuracy={}
Abbu=Ali=Monis=NahidMamu=TipuMamu=Wali=0
# model = load_model('/content/gdrive/My Drive/ExtendedFamilyClassifier/Models/Alexnet/AlexNetModel9-14-0.89.hdf5')

for k in range(len(file_name_prefix_list)):
    folder=sub_dir_path_list[k]
    class_name= sub_dir_name_list[k]
    file_num= len(os.listdir(folder))
    prefix= file_name_prefix_list[k]
    
    for i in range(file_num):
        
        file_name=prefix+str(i)+".jpg"
        file_path = os.path.join(os.path.join(folder, file_name))
        #Input(=input_shape; step 1.1) image size and test(=test_image) image size should be same
#         test_image = image.load_img(file_path, target_size = (28, 28))
        test_image = image.load_img(file_path, target_size = (224, 224,3))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model.predict(test_image)
        pred_mat=list(result[0])
        
        if pred_mat.index(max(pred_mat)) == 0 and prefix==sub_dir_name_list[0][0:len(prefix)]:
#             prediction = class_name[0] #Model 14: 36/41
            Abbu= Abbu +1
            Accuracy[class_name]=Abbu/file_num
#             count=count+1
#             print(prediction) 
        elif pred_mat.index(max(pred_mat))==1 and prefix==sub_dir_name_list[1][0:len(prefix)]:
#             prediction = class_name[1] #Model 14: 9/15
            Ali= Ali +1
            Accuracy[class_name]=Ali/file_num
#             count=count+1
#             print(prediction)
        elif pred_mat.index(max(pred_mat))==2 and prefix==sub_dir_name_list[2][0:len(prefix)]: 
            prediction = 'Monis' #Model 14: 38/66
            Monis= Monis +1
            Accuracy[class_name]=Monis/file_num
#             count=count+1
#             print(prediction)
        elif pred_mat.index(max(pred_mat))==3 and prefix==sub_dir_name_list[3][0:len(prefix)]:
            prediction = 'Nahid Mamu' #Model 14: 9/17
            NahidMamu= NahidMamu +1
            Accuracy[class_name]=NahidMamu/file_num
#             print(prediction)                    
#             count=count+1
        elif pred_mat.index(max(pred_mat))==4 and prefix==sub_dir_name_list[4][0:len(prefix)]:
            prediction = 'Tipu Mamu' #Model 14 9/12
            TipuMamu= TipuMamu +1
            Accuracy[class_name]=TipuMamu/file_num
#             print(prediction)
#             count=count+1
        elif pred_mat.index(max(pred_mat))==5 and prefix==sub_dir_name_list[5][0:len(prefix)]:
            prediction = 'Wali' #Model 14: 9/16
            Wali=Wali+1
            Accuracy[class_name]=Wali/file_num
#             print(prediction)
#             count=count+1
#         print(max(list(result[0])))
       

            


print(Accuracy)

#VGGModel1-Test:{'Abbu': 0.926829268292683, 'Ali': 0.8666666666666667, 'Monis': 0.8947368421052632, 'Nahid Mamu': 
#                   0.7647058823529411, 'Tipu Mamu': 0.5833333333333334, 'Wali': 1.0}---Avg=0.8394

#VGGModel1-Train:{'Abbu': 0.9714285714285714, 'Ali': 0.9571428571428572, 'Monis': 1.0, 'NahidMamu': 0.9436619718309859, 
#                   'TipuMamu': 0.971830985915493, 'Wali': 0.9701492537313433}---Avg=0.96903561

